Handling missing irrigation data

Proxy approach
- Calculate NDMI changes between consecutive readings
- Set thresholds for significant moisture changes
- Identify moisture spikes - check with precipitation levels

In [1]:
import rasterio
import numpy as np
import pandas as pd
from datetime import datetime
import glob
import os

# Read all NDMI files in chronological order

def read_tif_files(directory_path):
    # Get all TIF files
    tif_files = glob.glob(os.path.join(directory_path, '*NDMI*.tif'))
    
    # Store data in a dictionary with filenames as keys
    ndmi_data = {}
    for file in tif_files:
        filename = os.path.basename(file)
        with rasterio.open(file) as src:
            ndmi_data[filename] = src.read(1)  # Read the first band
            
    return ndmi_data, src.meta

Calculate changes between consecutive dates

In [2]:
def calculate_ndmi_changes(ndmi_data, precip_df):
    # Sort precipitation data by start_date to ensure chronological order
    precip_df = precip_df.sort_values('start_date')
    
    ndmi_changes = {}
    precip_values = {}
    
    # Iterate through precipitation data to get consecutive TIFF files
    for i in range(1, len(precip_df)):
        current_tiff = precip_df.iloc[i]['tiff_file']
        previous_tiff = precip_df.iloc[i-1]['tiff_file']
        
        if current_tiff in ndmi_data and previous_tiff in ndmi_data:
            change = ndmi_data[current_tiff] - ndmi_data[previous_tiff]
            ndmi_changes[current_tiff] = change
            precip_values[current_tiff] = precip_df.iloc[i]['precipitation_sum']
    
    return ndmi_changes, precip_values

Identify significant moisture changes and potential irrigation events

In [3]:
def identify_irrigation_events(ndmi_changes, precip_values, threshold=0.1):
    irrigation_events = {}
    
    for tiff_file, change in ndmi_changes.items():
        # Get precipitation for this period
        precip = precip_values.get(tiff_file, 0)
        
        # Create a mask for significant positive changes
        significant_increase = change > threshold
        
        # If there's a significant increase but low precipitation,
        # it's likely an irrigation event
        if precip < 20:  # Adjusted threshold for weekly precipitation
            irrigation_mask = significant_increase
            
            # Calculate approximate irrigation amount
            # Subtract the precipitation contribution from the NDMI change
            # Assuming linear relationship between NDMI change and water amount
            precipitation_contribution = precip / 100  # Convert mm to NDMI units
            adjusted_change = change - precipitation_contribution
            
            # Only consider positive changes after precipitation adjustment
            adjusted_change[adjusted_change < 0] = 0
            
            irrigation_amount = adjusted_change[irrigation_mask] * 100
            
            irrigation_events[tiff_file] = {
                'area_irrigated_pixels': np.sum(irrigation_mask),
                'average_amount_mm': np.mean(irrigation_amount) if len(irrigation_amount) > 0 else 0,
                'weekly_precipitation_mm': precip
            }
    
    return irrigation_events

Load weather data and combine everything

In [4]:
def analyze_irrigation_patterns(directory_path, precip_df):
    # Read NDMI data
    ndmi_data, metadata = read_tif_files(directory_path)
    
    # Calculate NDMI changes with precipitation data
    ndmi_changes, precip_values = calculate_ndmi_changes(ndmi_data, precip_df)
    
    # Identify irrigation events
    irrigation_events = identify_irrigation_events(ndmi_changes, precip_values)
    
    # Convert results to DataFrame
    results_df = pd.DataFrame.from_dict(irrigation_events, orient='index')
    
    # Add dates from precipitation DataFrame
    results_df = results_df.merge(
        precip_df[['tiff_file', 'start_date', 'end_date']], 
        left_index=True, 
        right_on='tiff_file'
    )
    
    return results_df

In [5]:
ndmi_directory = "data/NDMI_Weekly_Exports/NDMI_Weekly_Exports_2019_2024"

# Example of how your precipitation DataFrame might look
precip_df = pd.read_csv("data/precipitation_sums.csv")

In [6]:
results = analyze_irrigation_patterns(ndmi_directory, precip_df)

In [7]:
results.head()

,area_irrigated_pixels,average_amount_mm,weekly_precipitation_mm,tiff_file,start_date,end_date
1,2,5.146973,5.24,NDMI_2019-01-15_to_2019-01-22.tif,2019-01-15,2019-01-22
2,150,15.401348,3.60,NDMI_2019-02-05_to_2019-02-12.tif,2019-02-05,2019-02-12
3,214,19.548165,0.57,NDMI_2019-02-12_to_2019-02-19.tif,2019-02-12,2019-02-19
5,0,0.000000,2.31,NDMI_2019-03-12_to_2019-03-19.tif,2019-03-12,2019-03-19
6,0,0.000000,0.89,NDMI_2019-03-19_to_2019-03-26.tif,2019-03-19,2019-03-26


In [8]:
results.to_csv('data/irrigation_estimates.csv', index=False)